## Assignment 5: Advance pandas
Let's continue to use the dataset from the wine magazine used in notebook 'Colab_Lec06_advanced-pandas' to practice data transformation, grouping, and sorting.

<img src="https://secure.static.meredith.com/crt/store/covers/magazines/nmo/9826_l.jpg">

In [1]:
csvurl="https://gist.githubusercontent.com/clairehq/" + \
        "79acab35be50eaf1c383948ed3fd1129/raw/407a02139ae1e134992b90b4b2b8c329b3d73a6a/winemag-data-130k-v2.csv"
import pandas as pd
wine = pd.read_csv(csvurl)
wine.drop(wine.columns[0], axis="columns", inplace=True)
wine.head()

,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


#### Question 1: ####  
What is the mean of the points column?

In [3]:
# Your code here . . .
#So to find the mean we can just do .mean

mean_points = wine['points'].mean()
print("The mean answer is:", mean_points)

The mean answer is: 88.43403716087269


#### Question 2: ####  
How many countries are present in this dataset? (Only count each country once)

In [4]:
# Your code here . . .
how_many_countries = wine['country'].nunique()
print("The answer is:", how_many_countries)

The answer is: 41


#### Question 3: ####
How many times does each country appeared in this dataset? Show each country and the corresponding count (show counts in ascending order)

In [5]:
# Your code here . . .
country_counts = wine['country'].value_counts()
print("The answer is:")
print(country_counts)
#Automation by google and then click on tab

The answer is:
country
US                        27177
France                    11174
Italy                     10005
Spain                      3409
Portugal                   2963
Chile                      2258
Argentina                  1907
Austria                    1635
Australia                  1177
Germany                    1051
South Africa                737
New Zealand                 733
Israel                      259
Greece                      242
Canada                      108
Bulgaria                     68
Romania                      67
Hungary                      61
Uruguay                      61
Croatia                      44
Turkey                       43
Slovenia                     39
Georgia                      37
England                      36
Mexico                       31
Brazil                       31
Moldova                      30
Lebanon                      20
Morocco                      11
Peru                          8
Serbia           

#### Question 4: ####
Create a variable `adjusted_price` containing the adjusted price which is the price subtracted by the average price. *This is called **"centering" transformation** - a method commonly used in the preprocessing step before applying various machine learning algorithms.*

In [6]:
# Your code here . . .

#Lets understnad what this problem really want us to do

#We are funding the adjusted price and that is subtracting from the individual price
#Pretty much the new value tell us how much or less expensive the wine is compare to the average

#This question 4 is so crucial because of improve on performance
average_price = wine['price'].mean()
adjusted_price = wine['price'] - average_price
print("The answer is:")
print(adjusted_price)

The answer is:
0              NaN
1       -20.232932
2       -21.232932
3       -22.232932
4        29.767068
           ...    
65494     9.767068
65495   -13.232932
65496   -15.232932
65497    -4.232932
65498   -25.232932
Name: price, Length: 65499, dtype: float64


#### Question 5: ####
What is the title of the wine that has the highest points-to-price ratio in the dataset?

In [8]:
# Your code here . .
highest_ration = wine['points'] / wine['price']
max_ratio = highest_ration.idxmax()
wine.loc[max_ratio, 'title']

'Bandit NV Merlot (California)'

#### Question 6: ####
Create a series `flavor_counts` that contains two values: the number of wines that has the word "tart" in the `description` column and the number of wines that has the word "berries" in the `description` column. The index of the Series should be "Tart" and "Berries" for the corresponding values.

In [12]:
# Look for the word "tart", not case sensitive, not a substring of another word.
# Same applies to the word "berries"
# Your code here . . .

#We have to understand thatpd.series is storing two count in a series with the required index
tart_couunt = wine['description'].str.contains('tart', case=False).sum()
berries_count = wine['description'].str.contains('berries', case=False).sum()


flavor_counts = pd.Series([tart_couunt, berries_count], index=['Tart', 'Berries'])
print("The answer is:")
print(flavor_counts)

The answer is:
Tart       4424
Berries    3512
dtype: int64


#### Question 7: ####
Let's convert the points into simple star ratings. A score of 90 or higher counts as 3 stars, a score of at least 80 but less than 90 is 2 stars. Any other score is 1 star.

Also, any wines from France should automatically get 3 stars, regardless of points.

Add this new column `star_ratings` to the dataframe with the number of stars for each wine in the dataset.

In [16]:
# Your code here . . .

#so i pretty much create a function with the simple if statement,
#if the wine is from france give them a 3
#if wine is 90 or more give it a 3 start
#if the wine has between 80 and 89 point 2 starts
#less than 80 1 star

def convert_points_to_stars(row):
    if row['country'] == 'France':
        return 3

    if row['points'] >= 90:
        return 3
    elif row['points'] >= 80:
        return 2
    else:
        return 1

wine['star_ratings'] = wine.apply(convert_points_to_stars, axis=1)
print("The answer is:")
print(wine[['country', 'points', 'star_ratings']].head())





The answer is:
    country  points  star_ratings
0     Italy      87             2
1  Portugal      87             2
2        US      87             2
3        US      87             2
4        US      87             2


#### Question 8: ####
Who are the most common wine reviewers in the dataset? Create a Series whose index is the taster_twitter_handle category from the dataset, and whose values count how many reviews each person wrote.

In [17]:
# Your code here . . .
most_common_reviewers = wine['taster_twitter_handle'].value_counts()
print("The answer is:")
print(most_common_reviewers)

The answer is:
taster_twitter_handle
@vossroger          13045
@wineschach          7752
@kerinokeefe         5313
@paulgwine           4851
@vboone              4696
@mattkettmann        3035
@JoeCz               2605
@wawinereport        2358
@gordone_cellars     2032
@AnneInVino          1769
@laurbuzz             938
@suskostrzewa         593
@worldwineguys        465
@bkfiona               11
@winewchristina         4
Name: count, dtype: int64


#### Question 9: ####
What combination of countries and varieties are most common? Create a Series whose index is a MultiIndexof {country, variety} pairs. For example, a pinot noir produced in the US should map to {"US", "Pinot Noir"}. Sort the values in the Series in descending order based on wine count.

In [18]:
# Your code here . . .
most_common_combination = wine.groupby(['country', 'variety']).size().sort_values(ascending=False)
print("The answer is:")
print(most_common_combination)


The answer is:
country  variety                 
US       Pinot Noir                  4918
         Cabernet Sauvignon          3649
         Chardonnay                  3412
France   Bordeaux-style Red Blend    2380
Italy    Red Blend                   1870
                                     ... 
         Torbato                        1
         Vespaiolo                      1
         Vespolina                      1
         Vitovska                       1
Uruguay  Tempranillo-Tannat             1
Length: 1304, dtype: int64


#### Question 10 #####
Create a Series whose index is reviewers and whose values is the average score given out by that reviewer. Hint: you will need the `taster_name` and `points` columns.

In [19]:
# Your code here . . .
the_average_score = wine.groupby('taster_name')['points'].mean()
print("The answer is:")
print(the_average_score)

#lets just use the groupby function, this means that all the wine review are group together
#then average calct point

The answer is:
taster_name
Alexander Peartree    86.014286
Anna Lee C. Iijima    88.380506
Anne Krebiehl MW      90.587903
Carrie Dykes          86.644444
Christina Pickard     89.500000
Fiona Adams           87.090909
Jeff Jenssen          88.273504
Jim Gordon            88.604331
Joe Czerwinski        88.519770
Kerin O’Keefe         88.827969
Lauren Buzzeo         87.831557
Matt Kettmann         90.021087
Michael Schachner     86.904541
Mike DeSimone         89.030303
Paul Gregutt          89.095032
Roger Voss            88.678957
Sean P. Sullivan      88.666243
Susan Kostrzewa       86.408094
Virginie Boone        89.229557
Name: points, dtype: float64
